In [1]:
!pip install -q diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline
from diffusers import UniPCMultistepScheduler
model_name= "monster-labs/control_v1p_sd15_qrcode_monster" #@param {type:"string"}
#replace the above pretrained model name with any you'd like to use
controlnet = ControlNetModel.from_pretrained(model_name)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
	"runwayml/stable-diffusion-v1-5", controlnet=controlnet
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload() # no explicit tocuda()
pipe.enable_xformers_memory_efficient_attention()



The config attributes {'global_pool_conditions': False} were passed to ControlNetModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
!git clone https://github.com/TheOneTrueGuy/DigitalDayDream.git
!unzip -d /content/ /content/DigitalDayDream/yinyang.zip

In [ ]:
# run this first then make anim in next cell
import torch
#from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
#from diffusers import DiffusionPipeline
import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image
import math
class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)


#pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()



In [ ]:

zip_name = "test1.zip" #@param {type:"string"}
animation_name = "test1.mp4" #@param {type:"string"}
control_image_path = "/content/yinyang/" #@param {type:"string"}
csv_prompts = "prompty prompt from promptland, frogs chasing dogs on logs in bogs with hogs, bizarre butterfly shimmering illusion of yin and yang" #@param {type:"string"}
negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality" #@param {type:"string"}
change_prompt_every = 10 #@param {type:"integer"}
number_of_control_images = 23 #@param {type:"integer"}
max_frames = 115 #@param {type:"integer"}
prompt_list=csv_prompts.split(",")
plist_len=len(prompt_list)

outdir = dt.datetime.now().strftime("%Y%m%d%H%M")
# if outdir exists add 4 random digits to it:
if os.path.exists(outdir):
    # Generate 4 random digits
    random_digits = str(random.randint(1000, 9999))

    # Create a new output directory with the random digits appended
    new_outdir = outdir + random_digits

    os.mkdir(new_outdir)
    outdir=new_outdir
    print("Created new output directory:", new_outdir)
else:
    os.system(f"mkdir {outdir}")
print("date time now:" + outdir)


xl=0
xr=2
xz=0
pc=0
total=max_frames
generator = torch.Generator(device="cpu").manual_seed(2)
#control_image= load_image("grill5.png").convert("RGB") # used for a single fixed frame control
prompt=prompt_list[0]

for xn in range(0,total,1):#must equal xz
  if xn%100==99: torch.cuda.empty_cache() # change to 100 and 99 for 512x512, 50 & 49 for
  if xn%change_prompt_every==change_prompt_every-1: # this freaky bit of code allows you to change the overlap between full iterations of the control images
    pc=pc+1
    prompt=prompt_list[(pc%change_prompt_every)%plist_len]
    print(pc)


  print(prompt) # so you can see when it changes over

  xr=1+(xn%number_of_control_images)
  naym = control_image_path +str(xr).zfill(5) + ".png" # change this to match the name of your sequence of images.
  control_image= load_image(naym).convert("RGB")

  image = pipe(
    prompt,
    control_image,
    negative_prompt=negative_prompt,
    generator=generator,
    num_inference_steps=5,
  )[0]


  out2=outdir+"/" + str(xz).zfill(4) + ".png"
  xz+=1
  print(str(xz) + " of " + str(total))
  image[0].save(out2) # save animation frame

os.system(f"ffmpeg -framerate 24.5 -i %04d.png -ac 2 -pix_fmt yuv420p -vsync 2 {animation_name} ")
os.system(f"cp {animation_name}  /content/drive/MyDrive")
#if the above errors because you didn't use google drive you can still download from the left side-list.
if zip_name !="none":
  os.system(f"zip -r {zip_name} {outdir}")
  os.system(f"cp {zip_name}  /content/drive/MyDrive")


In [ ]:
#run this cell after starting cell above if you don't want to burn
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()